## Prepare Dataset

Let's load a new dataset, for example, the `load_digits` dataset from `sklearn.datasets` and split this dataset into training and testing sets (`X_train_new`, `X_test_new`, `y_train_new`, `y_test_new`).


In [16]:
from sklearn.datasets import load_digits

print("load_digits imported successfully.")

load_digits imported successfully.


The `load_digits` is imported. the digits dataset is loaded and split into training and testing sets using `train_test_split`, and then display the first few rows of `X_train_new` to confirm the data preparation.


In [17]:
X_new, y_new = load_digits(return_X_y=True, as_frame=True)
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(
    X_new, y_new, test_size=0.2, random_state=201
)
X_train_new.head(n=5)

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7
1331,0.0,0.0,6.0,16.0,16.0,6.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,7.0,12.0,0.0,0.0,0.0,0.0
1628,0.0,0.0,5.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,16.0,1.0,0.0,0.0,0.0
22,0.0,0.0,8.0,16.0,5.0,0.0,0.0,0.0,0.0,1.0,...,3.0,0.0,0.0,0.0,7.0,12.0,12.0,12.0,13.0,1.0
533,0.0,0.0,5.0,11.0,16.0,16.0,5.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,8.0,16.0,0.0,0.0,0.0,0.0
1787,0.0,0.0,10.0,16.0,15.0,1.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,6.0,13.0,10.0,4.0,0.0,0.0


## Installing Ray
Let's install the ray library

In [18]:
!pip install ray

## Initializing Ray
After the `ray` library has been installed, Ray is initialized and then the training and testing datasets is placed into Ray's object store.



In [19]:
import ray

if ray.is_initialized():
    ray.shutdown()

ray.init()

X_train_new_ref = ray.put(X_train_new)
X_test_new_ref = ray.put(X_test_new)
y_train_new_ref = ray.put(y_train_new)
y_test_new_ref = ray.put(y_test_new)

print("New training and testing data placed into Ray's object store.")

2025-12-02 20:22:59,716	INFO worker.py:2023 -- Started a local Ray instance.


New training and testing data placed into Ray's object store.


## Creating new Ray remote function
Let's now define a new Ray remote function for a classifier model `train_and_score_model`. This function will use `RandomForestClassifier` and calculate `accuracy_score`.


In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time

@ray.remote
def train_and_score_classifier_model(
    train_set_ref: pd.DataFrame,
    test_set_ref: pd.DataFrame,
    train_labels_ref: pd.Series,
    test_labels_ref: pd.Series,
    n_estimators: int,
) -> tuple[int, float]:
    start_time = time.time()  # measure wall time for single model training

    model = RandomForestClassifier(n_estimators=n_estimators, random_state=201)
    model.fit(train_set_ref, train_labels_ref)
    y_pred = model.predict(test_set_ref)
    score = accuracy_score(test_labels_ref, y_pred)

    time_delta = time.time() - start_time
    print(
        f"n_estimators={n_estimators}, accuracy={score:.4f}, took: {time_delta:.2f} seconds"
    )

    return n_estimators, score

print("train_and_score_classifier_model remote function defined.")

train_and_score_classifier_model remote function defined.


## Parallel Execution
Let's now create a `run_parallel_new_classifier` function which will launch multiple `train_and_score_classifier_model.remote` tasks in parallel, using the newly placed data references (`X_train_new_ref`, `X_test_new_ref`, `y_train_new_ref`, `y_test_new_ref`). It will then use `ray.get()` to collect the results (n_estimators and accuracy scores) and measure the total wall time for the parallel execution.



In [26]:
def run_parallel_new_classifier(n_models: int) -> list[tuple[int, float]]:
    results_ref = [
        train_and_score_classifier_model.remote(
            train_set_ref=X_train_new_ref,
            test_set_ref=X_test_new_ref,
            train_labels_ref=y_train_new_ref,
            test_labels_ref=y_test_new_ref,
            n_estimators=8 + 4 * j,
        )
        for j in range(n_models)
    ]
    return ray.get(results_ref)

print("run_parallel_new_classifier function defined.")

run_parallel_new_classifier function defined.


Now let's run the parallel model training for the dataset using `run_parallel_new_classifier` . Let's use the `%%time` magic command to measure the wall time of this execution.



Let's use the `NUM_MODELS` constant for the number of models to train.



In [27]:
NUM_MODELS = 20

In [31]:
%%time
classifier_accuracy_scores = run_parallel_new_classifier(n_models=NUM_MODELS)

(train_and_score_classifier_model pid=4053) n_estimators=8, accuracy=0.9250, took: 0.07 seconds
CPU times: user 64.1 ms, sys: 9.48 ms, total: 73.6 ms
Wall time: 3.64 s


## Results
After the parallel training of the classifier models is complete, the results are stored in `classifier_accuracy_scores`, Let's analyze these scores to find the best performing model. This involves finding the tuple with the highest accuracy score (the second element) and then printing the corresponding `n_estimators` and accuracy.



In [32]:
from operator import itemgetter
best_classifier = max(classifier_accuracy_scores, key=itemgetter(1))
print(f"Best classifier model: accuracy={best_classifier[1]:.4f}, n_estimators={best_classifier[0]}")

Best classifier model: accuracy=0.9722, n_estimators=76


The final step is to shut down the Ray runtime to release resources.



In [33]:
ray.shutdown()

## Summary:

The implementation involved loading the `load_digits` dataset, splitting it, and storing it in Ray's object store. A new Ray remote function, `train_and_score_classifier_model`, was defined to train `RandomForestClassifier` models in parallel for various `n_estimators` values. This parallel execution, involving 20 models, took approximately 10.1 seconds. The best model found achieved an accuracy of 0.9722 with 76 estimators.

### Data Analysis Key Findings
*   The `load_digits` dataset was successfully loaded and split into training and testing sets, with `X_train_new` showing 64 pixel features.
*   Ray was successfully initialized, and the new dataset components (`X_train_new`, `X_test_new`, `y_train_new`, `y_test_new`) were placed into Ray's object store using `ray.put()`.
*   A remote function, `train_and_score_classifier_model`, was defined to train `RandomForestClassifier` models, taking object references for data and returning the `n_estimators` and accuracy.
*   Parallel execution of 20 `RandomForestClassifier` models with varying `n_estimators` using `run_parallel_new_classifier` took approximately 10.1 seconds (wall time).
*   The best performing model achieved an accuracy of 0.9722 with `n_estimators=76`. For context, a model with `n_estimators=8` yielded an accuracy of 0.9250.

### Insights or Next Steps
*   The significant improvement in accuracy from 0.9250 (n\_estimators=8) to 0.9722 (n\_estimators=76) suggests that increasing `n_estimators` generally improves model performance for this dataset, up to a certain point. Further hyperparameter tuning for `RandomForestClassifier` (e.g., `max_depth`, `min_samples_split`) could be explored in parallel.
*   The parallel execution framework with Ray efficiently trains multiple models simultaneously. This approach can be extended to perform more extensive hyperparameter searches or cross-validation for robust model selection.
